问给n个元素，每个元素有m个不同的topic，设计一个divesifier让相邻的k个不能有重复的topic，并且尽量保持原来的顺序。

# Ranking diversification
Given n items in **arr**, their relevance score **rel**, and their groups in **group**, find the best ranking of k<=n items. If not possible, output None.
The diversification rule says no **m** consecutive items belong to the same group. 
The best ranking is defined by NDCG score.
If no relevance score is available, try to minimize the rank of each item picked.

```Python
Example 1:
    Input:
        arr = ["tv", "speaker", "socks", "headphone", "shirt", "coffee", "monitor", "keyboard", "tea", "jeans", "tshirt"]
        group = ["electronics", "electronics", "fashion", "electronics", "fashion", "food", "electronics", "electronics", "food", "fashion", "fashion"]
        k = 8, m = 3
    Output:
        ["tv", "socks", "coffee", "speaker", "shirt", "tea", "headphone", "jeans"]
    Explanation:
        There are no any 3 consecutive items belong to the same group.
```

In [1]:
arr = ["tv", "speaker", "socks", "headphone", "shirt", "coffee", "monitor", "keyboard", "tea", "jeans", "tshirt"]
group = ["electronics", "electronics", "fashion", "electronics", "fashion", "food", "electronics", "electronics", "food", "fashion", "fashion"]

In [2]:
from typing import List
def Rerank(arr: List[str], group: List[str], m:int, k:int, rel: List[float]=None):
    assert len(arr)==len(group), "Sizes of items and groups must match!"

    item_map = {}
    idx = {}
    for i, v in enumerate(zip(arr, group)):
        if v[1] not in item_map:
            item_map[v[1]] = []
            idx[v[1]] = 0
        item_map[v[1]].append(i)
    # print(idx, item_map)
    res = []
    g = []
    while len(res) < k:
        # print(res, idx)
        cand = len(arr)
        for t,v in item_map.items():
            if t not in g[-m+1:] and idx[t]<len(v):
                cand = min(cand, v[idx[t]])
        if cand < len(arr):
            res.append(arr[cand])
            g.append(group[cand])
            idx[group[cand]] += 1
        else:
            return None
    return res
        
            
        

In [3]:
Rerank(arr, group, 3, 8)

['tv', 'socks', 'coffee', 'speaker', 'shirt', 'tea', 'headphone', 'jeans']

In [4]:
Rerank(arr, group, 3, 9)

In [5]:
def Rerank2(arr: List[str], group: List[str], m:int, k:int, rel: List[float]=None):
    def recur(res, visited, g_selected, l):
        # print(res, visited)
        if len(l)>0:
            return
        if len(res) == k:
            l.append(res)
        for item, g in zip(arr, group):
            if item not in visited and g not in g_selected[-m+1:]:
                visited[item] = True
                recur(res+[item], visited, g_selected+[g], l)
                del visited[item]
    l = []
    recur([], {}, [], l)
    if len(l)>0:
        return l[0]
    return None
                
    

In [6]:
Rerank2(arr, group, 3, 8)

['tv', 'socks', 'coffee', 'speaker', 'shirt', 'tea', 'headphone', 'jeans']

In [7]:
arr = ["coffee", "tv", "speaker", "socks", "headphone", "shirt", "monitor", "keyboard", "tea", "jeans", "tshirt"]
group = ["food", "electronics", "electronics", "fashion", "electronics", "fashion",  "electronics", "electronics", "food", "fashion", "fashion"]

In [8]:
Rerank(arr, group, 3, 8)

In [9]:
Rerank2(arr, group, 3, 8)

['tv', 'socks', 'coffee', 'speaker', 'shirt', 'tea', 'headphone', 'jeans']

In [10]:
import math
def Rerank3(arr: List[str], group: List[str], m:int, k:int, rel: List[float]=None):
    def recur(res, visited, g_selected, l):
        if len(res) == k:
            l.append(res)
        for item, g in zip(arr, group):
            if item not in visited and g not in g_selected[-m+1:]:
                visited[item] = True
                recur(res+[item], visited, g_selected+[g], l)
                del visited[item]
    l = []
    recur([], {}, [], l)
    rel_score = {}
    for item, s in zip(arr, rel):
        rel_score[item] = s
    
    if len(l)>0:
        max_score = 0
        res = None
        for x in l:
            s = 0
            for i in range(k):
                s += rel_score[x[i]]/math.log(i+2) * math.log(2)
            if s > max_score:
                res = x.copy()
                max_score = s
        return res
            
    return None
                

In [11]:
Rerank3(arr, group, 3, 8, [4,4,4,3,3,2,2,2,2,2,1])

['tv', 'socks', 'coffee', 'speaker', 'shirt', 'tea', 'headphone', 'jeans']

In [12]:
Rerank3(arr, group, 3, 8, [1,1,1,2,2,3,3,3,3,4,4])

['jeans',
 'monitor',
 'tea',
 'tshirt',
 'keyboard',
 'coffee',
 'shirt',
 'headphone']

In [13]:
Rerank3(arr, group, 4, 8, [1,1,1,2,2,3,3,3,3,4,4])